- 자연어 처리에 있어 제일 처음 수행해야 하는 기본적인 과정
- 전처리 결과와 품질에 따라 후처리및 결과의 성능차이가 커지므로 중요한 (+ 손이 많이 가는) 작업

In [1]:
# 전처리 모듈은 챗봇 엔진 내에서 자주 사용하기 때문에 클래스로 설계 해 둡니다

from konlpy.tag import Komoran
import pickle
import jpype


class Preprocess:
    def __init__(self, userdic=None):  # 생성자

        # ① 형태소 분석기 초기화
        self.komoran = Komoran(userdic=userdic)  # 사용자 정의 사전 파일의 경로 입력 가능

        # ② 제외할 품사
        # 참조 : https://docs.komoran.kr/firststep/postypes.html
        # 관계언 제거, 기호 제거
        # 어미 제거
        # 접미사 제거
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            'JX', 'JC',
            'SF', 'SP', 'SS', 'SE', 'SO',
            'EP', 'EF', 'EC', 'ETN', 'ETM',
            'XSN', 'XSV', 'XSA'
        ]

    # ③ 형태소 분석기 POS 태거
    #  이렇게 래퍼(wrapper) 를 만들어 두면 나중에 형태소 분석기 종류를 바꾸어도 동작할수 있다. (유지보수 측면 good)
    def pos(self, sentence):
        return self.komoran.pos(sentence)

    # ④ 불용어 제거 후, 필요한 품사 정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list



ModuleNotFoundError: No module named 'konlpy'

테스트

In [2]:
sent = "대출 조회좀해줘"

In [3]:
p = Preprocess(userdic = r'C:\Users\drago\OneDrive\문서\DevRoot\dataset\chatbot\user_dic.tsv')

In [4]:
pos = p.pos(sent)
pos

[('대출', 'NNG'),
 ('조회', 'NNG'),
 ('좀', 'MAG'),
 ('하', 'XSV'),
 ('아', 'EC'),
 ('주', 'VX'),
 ('어', 'EC')]

In [5]:
# 불용어 제거된 키워드 출력
ret = p.get_keywords(pos, without_tag=False)
ret

[('대출', 'NNG'), ('조회', 'NNG'), ('좀', 'MAG'), ('주', 'VX')]

In [6]:
ret = p.get_keywords(pos, without_tag=True)
ret

['대출', '조회', '좀', '주']

## 단어 사전 구축 및 시퀀스 생성

In [2]:
from tensorflow.keras import preprocessing
import pickle
import os
import pandas as pd

In [3]:
# 말뭉치 데이터 읽어오기
def read_corpus_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
    return data


# ① 말뭉치 데이터 가져오기
corpus_data = read_corpus_data(os.path.join(r'C:\DevRoot\Dropbox\Py03\09_chatbot\transaction\intent', 'corpus1.txt'))

In [4]:
print(len(corpus_data))
corpus_data[:10]

5682


[['안넝', '0'],
 ['ㅇㅏㄴ넝', '0'],
 ['아ㄴ넝', '0'],
 ['안ㄴㅓㅇ', '0'],
 ['안너ㅇ', '0'],
 ['얀뇽', '0'],
 ['ㅇㅑㄴㄴㅛㅇ', '0'],
 ['야ㄴㄴㅛㅇ', '0'],
 ['얀ㄴㅛㅇ', '0'],
 ['얀뇨ㅇ', '0']]

In [5]:
corpus = corpus_data[1:]
corpus

[['ㅇㅏㄴ넝', '0'],
 ['아ㄴ넝', '0'],
 ['안ㄴㅓㅇ', '0'],
 ['안너ㅇ', '0'],
 ['얀뇽', '0'],
 ['ㅇㅑㄴㄴㅛㅇ', '0'],
 ['야ㄴㄴㅛㅇ', '0'],
 ['얀ㄴㅛㅇ', '0'],
 ['얀뇨ㅇ', '0'],
 ['안녕해', '0'],
 ['ㅇㅏㄴㄴㅕㅇㅎㅐ', '0'],
 ['아ㄴㄴㅕㅇㅎㅐ', '0'],
 ['안ㄴㅕㅇㅎㅐ', '0'],
 ['안녀ㅇㅎㅐ', '0'],
 ['안녕ㅎㅐ', '0'],
 ['안녕하십니까', '0'],
 ['ㅇㅏㄴㅕㅇㅎㅏㅅㅣㅂㄴㅣㄲㅏ', '0'],
 ['아ㄴㄴㅕㅇㅎㅏㅅㅣㅂㄴㅣㄲㅏ', '0'],
 ['안ㄴㅕㅇㅎㅏㅅㅣㅂㄴㅣㄲ ㅏ', '0'],
 ['안녀ㅇㅎㅏㅅㅣㅂㄴㅣㄲㅏ', '0'],
 ['안녕ㅎㅏㅅㅣㅂㄴㅣㄲㅏ', '0'],
 ['안녕하ㅅㅣㅂㄴㅣㄲㅏ', '0'],
 ['안녕하시ㅂㄴㅣㄲㅏ', '0'],
 ['안녕하십ㄴㅣㄲㅏ', '0'],
 ['안녕하십니ㄲㅏ', '0'],
 ['안녕하시오', '0'],
 ['ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅣㅇㅗ', '0'],
 ['아ㄴㄴㅕㅇㅎㅏㅅㅣㅇㅗ', '0'],
 ['안ㄴㅕㅇㅎㅏㅅㅣㅇㅗ', '0'],
 ['안녀ㅇㅎㅏㅅㅣㅇㅗ', '0'],
 ['안녕ㅎㅏㅅㅣㅇㅗ', '0'],
 ['안녕하ㅅㅣㅇㅗ', '0'],
 ['안녕하시ㅇㅗ', '0'],
 ['안녕이라고 해도되', '0'],
 ['ㅇㅏㄴㄴㅕㅇㅇㅣㄹㅏㄱㅗㅎㅐㄷㅗㄷㅗㅣ', '0'],
 ['아ㄴㄴㅕㅇㅇㅣㄹㅏㄱ ㅗㅎㅐㄷㅗㄷㅗㅣ', '0'],
 ['안ㄴㅕㅇㅇㅣㄹㅏㄱㅗㅎㅐㄷㅗㄷㅗㅣ', '0'],
 ['안녀ㅇㅇㅣㄹㅏㄱㅗㅎㅐㄷㅗㄷㅗㅣ', '0'],
 ['안녕ㅇㅣㄹㅏㄱㅗㅎㅐㄷㅗㄷㅗㅣ', '0'],
 ['안녕이ㄹㅏㄱㅗㅎㅐㄷㅗㄷㅗㅣ', '0'],
 ['안녕이라ㄱㅗㅎㅐㄷㅗㄷㅗㅣ', '0'],
 ['안녕이라고ㅎㅐㄷㅗㄷㅗㅣ', '0'],
 ['안녕이라고해ㄷㅗㄷㅗㅣ', '0'],
 ['안녕이라고해도ㄷㅗㅣ', '0'],
 ['안녕이라고해도도ㅣ', '0'],
 ['안녕이라할게', '0'],
 ['ㅇㅏㄴㄴㅕㅇㅇㅣㄹㅏㅎㅏㄹㄱㅔ', '0'],
 ['아ㄴㄴㅕㅇㅇㅣㄹ

In [6]:
# ② 위에서 불러온 말뭉치 데이터 리스트에서 문장을 하나씩 불러와 POS 테깅  (★시간걸림★)
p = Preprocess()
dic = []
for c in corpus:
    li = []
    li = c[0].split(',')
    dic.append(li[0])


for i in dic:
    pos = p.pos(i)
pos



[('분실', 'NNG'),
 ('정보', 'NNG'),
 ('보', 'VV'),
 ('자', 'EC'),
 ('보', 'VX'),
 ('자', 'EC'),
 ('어디', 'NP'),
 ('보', 'VV'),
 ('자', 'EC')]

In [7]:
print(len(dic))
dic[:20]

5681


['ㅇㅏㄴ넝',
 '아ㄴ넝',
 '안ㄴㅓㅇ',
 '안너ㅇ',
 '얀뇽',
 'ㅇㅑㄴㄴㅛㅇ',
 '야ㄴㄴㅛㅇ',
 '얀ㄴㅛㅇ',
 '얀뇨ㅇ',
 '안녕해',
 'ㅇㅏㄴㄴㅕㅇㅎㅐ',
 '아ㄴㄴㅕㅇㅎㅐ',
 '안ㄴㅕㅇㅎㅐ',
 '안녀ㅇㅎㅐ',
 '안녕ㅎㅐ',
 '안녕하십니까',
 'ㅇㅏㄴㅕㅇㅎㅏㅅㅣㅂㄴㅣㄲㅏ',
 '아ㄴㄴㅕㅇㅎㅏㅅㅣㅂㄴㅣㄲㅏ',
 '안ㄴㅕㅇㅎㅏㅅㅣㅂㄴㅣㄲ ㅏ',
 '안녀ㅇㅎㅏㅅㅣㅂㄴㅣㄲㅏ']

In [13]:
# 테스트 과정에서 우리가 사용한 단어가 사전에 존재하지 않을 수 있습니다. 
# 이 경우 챗봇 엔진에서는 OOV(Out-Of-Vocabulary) 처리를 하지만
# 품질 향상을 위해 단어 사전을 업데이트 하는 것이 좋습니다.  corpus.txt 파일에 해당 단어를 사용하는 말뭉치를 업데이트하여 
# 다시 단어 사전을 생성해주면 됩니다.
# 단어 사전을 만드는데 시간이 꽤 오래 걸립니다.   말뭉치 데이터를 추가할때 데이터 양식에 맞지 않으면 오류가 날수 있으니
# 반드시 \t 를 사용해 데이터를 컬럼에 맞게 추가해야 합니다

In [8]:
#  ③ Tokenizer 를 이용해 위에서 만든 dict 를 단어 인덱스 딕셔너리 (word_index) 데이터로 만듭니다
# 사전의 첫번 째 인덱스에는 OOV 사용
tokenizer = preprocessing.text.Tokenizer(oov_token='OOV')  # 어휘에 없으면 'OOV' 로 token 대체
tokenizer.fit_on_texts(dic)
word_index = tokenizer

In [9]:
Word_index = tokenizer.word_index
Word_index

{'OOV': 1,
 '분실조회': 2,
 '내계좌조회': 3,
 '대출조회': 4,
 '내대출조회': 5,
 '적금조회': 6,
 '내적금조회': 7,
 '계좌조회': 8,
 '분실조회좀': 9,
 '계좌조회좀': 10,
 '내계좌조회좀': 11,
 '대출조회좀': 12,
 '내대출조회좀': 13,
 '적금조회좀': 14,
 '내적금조회좀': 15,
 '하고십어': 16,
 '하고십아': 17,
 '하고십오': 18,
 '하고싶나': 19,
 '말': 20,
 '납부하려면': 21,
 '하고싶엉': 22,
 '하고시퍼': 23,
 '해주게': 24,
 '해주싶쇼': 25,
 '해주고': 26,
 '제발': 27,
 '해주면': 28,
 '하고싶아': 29,
 '하고': 30,
 '해고': 31,
 '하구': 32,
 '하고싶니': 33,
 '하고싶노': 34,
 '하고싶내': 35,
 '하고싶네': 36,
 '하고싶누': 37,
 '하고싶뉘': 38,
 '하고십우': 39,
 '하고십니': 40,
 '하고십나': 41,
 '하고십누': 42,
 '하고십노': 43,
 '하고십파': 44,
 '혜구싶어': 45,
 '하고싶이': 46,
 '하고싶시': 47,
 '해구싶엉': 48,
 '야돈보내려면': 49,
 '이르는': 50,
 '돈보내려면': 51,
 '돈보네려면': 52,
 '야돈보네려면': 53,
 '사람을': 54,
 '한다': 55,
 '납부': 56,
 '야납부할려면': 57,
 '말로': 58,
 '뜻으로': 59,
 '쓰인다': 60,
 '이체': 61,
 '어떡해': 62,
 '어떡해요': 63,
 '어떻해요': 64,
 '어떠해요': 65,
 '어떧해요': 66,
 '어떨까요': 67,
 '어찌해요': 68,
 '어쩔까': 69,
 '어쩔래': 70,
 '어쩔라': 71,
 '납부해': 72,
 '이': 73,
 '같은': 74,
 '납부해줘': 75,
 '납부해줘잉': 76,
 '새끼': 77,
 '속되게': 78,
 '쓰이는': 79,


In [10]:
# ④ 생성된 단어 인덱스 딕셔너리(word_index) 객체를 파일로 저장
f = open(os.path.join(r'C:\DevRoot\Dropbox\Py03\09_chatbot\transaction\intent',"chatbot_dict.bin"), "wb")  # binary 모드에선 encoding 옵션 필요없다
try:
    pickle.dump(Word_index, f)   # 파이썬 객체(인스턴스)를 피클로 저장하기
except Exception as e:
    print(e)
finally:
    f.close()


# pickle.dump로 객체(값)를 저장할 때는 open('james.p', 'wb')와 같이 파일 모드를 'wb'로 지정해야 합니다. 
# b는 바이너리(binary)를 뜻하는데, 
# 바이너리 파일은 컴퓨터가 처리하는 파일 형식